**Silver Curated Table**   
Join silver tables and create the `silver_curated` table for convenience cross-source reference.
---

In [0]:
-- ===== 1) SILVER CURATED TABLE: join  tables for convenience cross-source =====
USE CATALOG claims360_dev;
CREATE SCHEMA IF NOT EXISTS silver_curated;

CREATE OR REPLACE VIEW silver_curated.claims_current AS
WITH latest_277 AS (
  SELECT claim_id,
         MAX(ack_event_ts)                             AS last_277_event_ts,
         MAX_BY(status_code, ack_event_ts)             AS last_277_status_code,
         COUNT(*)                                      AS ack_event_count
  FROM silver.ack_277ca_events
  GROUP BY claim_id
)
SELECT
  ch.claim_id,
  ch.payer_id, p.payer_name, p.payer_type,
  ch.patient_id,
  ch.submission_date, ch.submission_ts,
  ch.total_charges, ch.expected_amount,
  ch.current_status, ch.current_status_ts,
  COALESCE(l277.last_277_event_ts, ch.last_277ca_event_ts)  AS last_277_event_ts,
  l277.last_277_status_code, l277.ack_event_count,
  COALESCE(pay.last_payment_ts, ch.last_835_event_ts)       AS last_835_event_ts,
  pay.net_paid_to_date, pay.paid_amount_to_date, pay.adjustments_to_date,
  pay.last_remit_id, pay.last_trace, pay.payment_event_count, pay.reason_codes_seen,
  ch.weekday_submitted, ch.is_weekend_submission,
  ch.source_system, ch._ingest_ts, ch._ingest_file,
  (ch.total_charges - COALESCE(pay.net_paid_to_date,0))     AS current_balance,
  DATEDIFF(current_date(), ch.submission_date)              AS days_since_submission
FROM silver.claim_headers ch
LEFT JOIN silver.dim_payer p                 ON p.payer_id = ch.payer_id
LEFT JOIN latest_277 l277                    ON l277.claim_id = ch.claim_id
LEFT JOIN silver.payments_835_by_claim pay   ON pay.claim_id = ch.claim_id;